# Importing PyDicom Module

In [1]:
import pydicom
from pydicom.filereader import dcmread
from pydicom.data import get_testdata_files
dirname='./unnet_valid/'

# Importing the validation dataset

In [2]:
import pandas as pd
df=pd.read_csv("valid-edemaonly.csv")

In [3]:
df.head()

,Unnamed: 0,Path,Sex,Age,Frontal/Lateral,AP/PA,Edema
0,0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0
1,1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0
2,3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,1.0
3,4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,0.0
4,5,CheXpert-v1.0-small/valid/patient64545/study1/...,Female,55,Frontal,AP,0.0


# Assigning female and males names randomly to the images in validation folder (202 images)

In [4]:
male_names=['Sunny','James','John','George','Harry']
female_names=['Kelsey','Mary','Lisa','Erica','Olivia']
import random
print(random.choice(female_names))

Erica


In [5]:
import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l,key = alphanum_key)

# Adding Study Description, Patient Age, Modality, Patient Gender and Name to the images for a good presentation in OHIF Viewer

In [9]:
import os
import re
from pydicom.dataelem import DataElement

i=0
for filename in natural_sort(os.listdir(dirname)):
    if filename[0:12]=='.ipynb_check':
        continue
    if re.match(r"^"+re.escape(filename[0:12]),df.Path[i][26:38], re.I):
        ds = dcmread(os.path.join(dirname,filename))
        if "StudyDescription" not in ds.dir():
            ds.add_new(0x00081030, 'LO', 'Chest X-Ray')
            ds.add_new
        if "PatientAge" not in ds.dir():
            ds.add_new(0x00101010, 'AS', str(df.loc[i,'Age']))
            ds.add_new
        if "Modality" not in ds.dir():
            ds.add_new(0x00080060, 'CS', 'X-Ray')
            ds.add_new
        ds.PatientID=str(filename[7:12])
        if df.loc[i,'Sex']=='Male':
            ds.PatientName=str(random.choice(male_names))
            ds.PatientSex=df.loc[i,'Sex']
        else:
            ds.PatientName=str(random.choice(female_names))
            ds.PatientSex=df.loc[i,'Sex']
        ds.save_as(os.path.join('./Edited_valid_dcm',filename))
    else:
        print('didnot match')
    i=i+1

# Checking to make sure if the changes are applied accordingly

In [11]:
#filename = get_testdata_files("./unnet_valid/patient64541-study1-view1_frontal.jpg.dcm")
ds = dcmread("./Edited_valid_dcm/patient64541-study1-view1_frontal.jpg.dcm")

In [12]:
ds

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.276.0.7230010.3.1.4.8323329.24087.1553621179.921042
(0008, 0020) Study Date                          DA: ''
(0008, 0030) Study Time                          TM: ''
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'X-Ray'
(0008, 0064) Conversion Type                     CS: 'WSD'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1030) Study Description                   LO: 'Chest X-Ray'
(0010, 0010) Patient's Name                      PN: 'George'
(0010, 0020) Patient ID                          LO: '64541'
(0010, 0030) Patient's Birth Date                DA: ''
(0010, 0040) Patient's Sex                       CS: 'Male'
(0010, 1010) Patient's Age                       AS: '73'
(0020, 000d) Study Instance UID 